In [ ]:
#!pip install torch

In [ ]:
#!pip install jiwer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#!pip install transformers==4.31.0


In [ ]:
#!pip install datasets==2.14.5

In [ ]:
#!pip install transformers[torch]

In [ ]:
#!pip install accelerate -U

In [29]:
#!pip install -U wandb

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import TrainingArguments, Trainer
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import load_metric
from ast import literal_eval

In [ ]:
language_code = 'ar'
language_name = 'arabic'

#base_model = "facebook/wav2vec2-xls-r-300m"
base_model = "/home/nmu1/Downloads/MTC/Train Model/40-1-1/output_models_Demo40_1-1/ar/facebookwav2vec2-xls-r-300m/finetuned-v4"
output_models_dir = f"/home/nmu1/Downloads/MTC/Train Model/40-2-2/output_models_Demo_random40-2-2/ar/facebookwav2vec2-xls-r-300m/finetuned-v17"
new_output_models_dir = f"/home/nmu1/Downloads/MTC/Train Model/40-2-2/output_models_Demo_random40-2-2/ar/facebookwav2vec2-xls-r-300m"

In [ ]:
train = pd.read_pickle(f"/home/nmu1/Downloads/MTC/Train Model/40-2-2/train_V_random-40-2-2.pbz2", compression='bz2')
dev = pd.read_pickle(f"/home/nmu1/Downloads/MTC/Train Model/40-2-2/adapt_V_random-40-2-2.pbz2", compression='bz2')

In [ ]:
print(train.column_names)

['input_values', 'input_length', 'labels']


In [ ]:
import pandas as pd
train = train.to_pandas()
print(train.head())
dev = dev.to_pandas()
print(dev.head())

                                        input_values  input_length  \
0  [0.06711511, 0.011185066, -0.012870867, -0.062...        131360   
1  [-0.00038258886, -0.00038258886, -0.0003825888...         99680   
2  [-0.00028004573, -0.00028004573, -0.0002800457...        116960   
3  [0.000388785, 0.000388785, 0.000388785, 0.0003...         96800   
4  [5.477058e-05, 5.477058e-05, 5.477058e-05, 5.4...        120320   

                                              labels  
0  [5, 2, 28, 7, 35, 28, 31, 10, 12, 34, 16, 34, ...  
1  [4, 7, 35, 28, 35, 12, 35, 31, 16, 34, 16, 28,...  
2  [20, 35, 31, 30, 3, 36, 28, 35, 16, 4, 28, 35,...  
3  [19, 35, 34, 14, 4, 28, 35, 19, 31, 13, 36, 28...  
4  [35, 12, 36, 34, 6, 28, 20, 34, 28, 14, 34, 35...  
                                        input_values  input_length  \
0  [-0.0010540755, -0.0010540755, -0.0010540755, ...         26112   
1  [0.0010558531, 0.0010558531, 0.0010558531, 0.0...         72192   
2  [-0.0016085549, -0.0016085549, -0.00

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(new_output_models_dir)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MTCDataset(Dataset):
    def __init__(self, df):
        super(Dataset, self).__init__()
        self.data = df[["input_values", "input_length", "labels"]].to_dict('records')
        
    def __len__(self):
        return len(self.data)
        

    def __getitem__(self, index):
        return self.data[index]

In [ ]:
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
        
    def __init__(self, processor: Wav2Vec2Processor, padding: Union[bool, str]):
        self.processor = processor
        self.padding = padding

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

/tmp/ipykernel_6039/24688115.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:

model = Wav2Vec2ForCTC.from_pretrained(
    base_model, 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True
)
  

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /home/nmu1/Downloads/MTC/Train Model/40-1-1/output_models_Demo40_1-1/ar/wav2vec2-large-xlsr-arabic/finetuned-v4 and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([38, 1024]) in the checkpoint and torch.Size([41, 1024]) in the model instantiated
- lm_head.bias: found shape torch.Size([38]) in the checkpoint and torch.Size([41]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    base_model, 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /home/nmu1/Downloads/MTC/Train Model/40-1-1/output_models_Demo40_1-1/ar/wav2vec2-large-xlsr-arabic/finetuned-v4 and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([38, 1024]) in the checkpoint and torch.Size([41, 1024]) in the model instantiated
- lm_head.bias: found shape torch.Size([38]) in the checkpoint and torch.Size([41]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_encoder()

In [ ]:
training_args = TrainingArguments(
  output_dir=new_output_models_dir,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=600,
  eval_steps=300,
  logging_steps=300,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
train_dataset = MTCDataset(train)
dev_dataset = MTCDataset(dev)

In [ ]:
from transformers import AdamW

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)

/home/nmu1/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=processor.feature_extractor,
    optimizers=(optimizer, None)  # Pass the optimizer here
)

In [ ]:
trainer.train()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


/home/nmu1/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/lib/python3/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
300,4.064700,1.052263,0.516978
600,0.268300,1.125242,0.533460
900,0.231300,1.160548,0.526356
1200,0.223000,1.095631,0.520152
1500,0.182400,1.198022,0.521857
1800,0.168800,1.222952,0.520625
2100,0.146800,1.258187,0.531092
2400,0.143200,1.303570,0.519583
2700,0.125300,1.304834,0.519820
3000,0.118200,1.256874,0.519110


/home/nmu1/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/lib/python3/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/nmu1/.local/lib/python3.10/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v

TrainOutput(global_step=6340, training_loss=0.3129705006391844, metrics={'train_runtime': 11723.3518, 'train_samples_per_second': 17.304, 'train_steps_per_second': 0.541, 'total_flos': 4.285596138748804e+19, 'train_loss': 0.3129705006391844, 'epoch': 10.0})

In [30]:
trainer.save_model(output_models_dir)